# AHP 1
6 - Maret - 2025

In [9]:
import cv2
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.spatial import KDTree

# Define Input and Output Directories
input_folder = r"D:\Disertasi Bu Sussi\20-Maret\citra 5\Fuzzy"
output_folder = r"D:\Disertasi Bu Sussi\20-Maret\citra 5\AHP-1"
os.makedirs(output_folder, exist_ok=True)

# Define Color Ranges for Red, Yellow, and Blue in HSV
lower_red1 = np.array([0, 120, 70])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 120, 70])
upper_red2 = np.array([180, 255, 255])
lower_yellow = np.array([15, 90, 90])
upper_yellow = np.array([35, 255, 255])
lower_blue = np.array([90, 50, 50])
upper_blue = np.array([130, 255, 255])

# Process Each Image in Input Folder
for image_name in os.listdir(input_folder):
    if image_name.endswith(".png") or image_name.endswith(".jpg"):
        image_path = os.path.join(input_folder, image_name)
        image = cv2.imread(image_path)
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        
        # Thresholding to Get Masks
        mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
        mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
        mask_red = cv2.bitwise_or(mask_red1, mask_red2)
        mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
        mask_blue = cv2.inRange(hsv, lower_blue, upper_blue)
        
        # Morphological Operations
        kernel = np.ones((5, 5), np.uint8)
        mask_red_cleaned = cv2.morphologyEx(mask_red, cv2.MORPH_CLOSE, kernel, iterations=3)
        mask_yellow_cleaned = cv2.morphologyEx(mask_yellow, cv2.MORPH_CLOSE, kernel, iterations=3)
        mask_blue_cleaned = cv2.morphologyEx(mask_blue, cv2.MORPH_CLOSE, kernel, iterations=3)
        
        # Find Contours
        contours_red, _ = cv2.findContours(mask_red_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_yellow, _ = cv2.findContours(mask_yellow_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours_blue, _ = cv2.findContours(mask_blue_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Compute Metrics
        berada_dalam_area_kuning = [any(cv2.boundingRect(cnt)[0] in cv2.boundingRect(y_cnt) for y_cnt in contours_yellow) for cnt in contours_red]
        distances_to_blue = [min([abs(cv2.pointPolygonTest(b_cnt, cv2.boundingRect(r_cnt)[:2], True)) for b_cnt in contours_blue]) if contours_blue else float('inf') for r_cnt in contours_red]
        Dikelilingi_Biru = [sum(1 for b_cnt in contours_blue if cv2.boundingRect(b_cnt)[0] in cv2.boundingRect(r_cnt)) >= 2 for r_cnt in contours_red]
        
        # Normalize Distance
        min_distance = min(distances_to_blue) if distances_to_blue else 1
        max_distance = max(distances_to_blue) if distances_to_blue else 1
        normalized_distances = [(d - min_distance) / (max_distance - min_distance) if max_distance > min_distance else 0 for d in distances_to_blue]
        
        # Compute AHP Score
        bobot_kriteria = {"Jarak_ke_Biru": 0.1038, "berada_dalam_area_kuning": 0.2310, "Dikelilingi_Biru": 0.6650}
        
        def hitung_skor_ahp(row):
            return (row["Jarak_ke_Biru"] * bobot_kriteria["Jarak_ke_Biru"] +
                    row["berada_dalam_area_kuning"] * bobot_kriteria["berada_dalam_area_kuning"] +
                    row["Dikelilingi_Biru"] * bobot_kriteria["Dikelilingi_Biru"])
        
        data_ahp = pd.DataFrame({"Jarak_ke_Biru": normalized_distances,
                                 "berada_dalam_area_kuning": berada_dalam_area_kuning,
                                 "Dikelilingi_Biru": Dikelilingi_Biru})
        data_ahp["Skor_Akhir"] = data_ahp.apply(hitung_skor_ahp, axis=1)
        
        # Modify Image
        threshold_ahp = data_ahp["Skor_Akhir"].median()
        image_modified = image.copy()
        for idx, cnt in enumerate(contours_red):
            color = (0, 255, 255) if idx < len(data_ahp) and data_ahp.iloc[idx]["Skor_Akhir"] > threshold_ahp else (0, 0, 255)
            cv2.drawContours(image_modified, [cnt], -1, color, thickness=cv2.FILLED)
            cv2.drawContours(image_modified, [cnt], -1, color, thickness=1)
        
        # Save Outputs
        output_image_path = os.path.join(output_folder, f"modified_{image_name}")
        output_excel_path = os.path.join(output_folder, f"ahp_results_{os.path.splitext(image_name)[0]}.xlsx")
        cv2.imwrite(output_image_path, image_modified)
        data_ahp.to_excel(output_excel_path, index=False)
        
        print(f"Processed {image_name} -> Image: {output_image_path}, Data: {output_excel_path}")

Processed merge_image5_obia0_result.png -> Image: D:\Disertasi Bu Sussi\20-Maret\citra 5\AHP-1\modified_merge_image5_obia0_result.png, Data: D:\Disertasi Bu Sussi\20-Maret\citra 5\AHP-1\ahp_results_merge_image5_obia0_result.xlsx
Processed merge_image5_obia25_result.png -> Image: D:\Disertasi Bu Sussi\20-Maret\citra 5\AHP-1\modified_merge_image5_obia25_result.png, Data: D:\Disertasi Bu Sussi\20-Maret\citra 5\AHP-1\ahp_results_merge_image5_obia25_result.xlsx
Processed merge_image5_obia50_result.png -> Image: D:\Disertasi Bu Sussi\20-Maret\citra 5\AHP-1\modified_merge_image5_obia50_result.png, Data: D:\Disertasi Bu Sussi\20-Maret\citra 5\AHP-1\ahp_results_merge_image5_obia50_result.xlsx


# AHP 2
6 - Maret - 2025

In [1]:
import cv2
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from scipy.spatial import KDTree

# Folder input dan output
input_folder = r"D:\Disertasi Bu Sussi\20-Maret\citra 5\Fuzzy"
output_folder = r"D:\Disertasi Bu Sussi\20-Maret\citra 5\AHP 1 Tahap"
os.makedirs(output_folder, exist_ok=True)

# Define Color Ranges for Red and Yellow in HSV
lower_red1 = np.array([0, 120, 70])
upper_red1 = np.array([10, 255, 255])
lower_red2 = np.array([170, 120, 70])
upper_red2 = np.array([180, 255, 255])
lower_yellow = np.array([15, 90, 90])
upper_yellow = np.array([35, 255, 255])

# AHP Weights
bobot_kriteria = {
    "Jarak_ke_Merah": 0.0833,
    "Sebagai_Penghubung": 0.1931,
    "Dikelilingi_Merah": 0.7235
}

def get_centroid(contour):
    M = cv2.moments(contour)
    if M["m00"] != 0:
        cx = int(M["m10"] / M["m00"])
        cy = int(M["m01"] / M["m00"])
        return (cx, cy)
    return None

def hitung_skor_ahp(row):
    return (
        row["Jarak_ke_Merah"] * bobot_kriteria["Jarak_ke_Merah"] +
        row["Sebagai_Penghubung"] * bobot_kriteria["Sebagai_Penghubung"] +
        row["Dikelilingi_Merah"] * bobot_kriteria["Dikelilingi_Merah"]
    )

# Proses Semua Gambar di Folder
image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
df_list = []
for image_file in image_files:
    image_path = os.path.join(input_folder, image_file)
    image = cv2.imread(image_path)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Thresholding
    mask_red1 = cv2.inRange(hsv, lower_red1, upper_red1)
    mask_red2 = cv2.inRange(hsv, lower_red2, upper_red2)
    mask_red = cv2.bitwise_or(mask_red1, mask_red2)
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    
    kernel = np.ones((5, 5), np.uint8)
    mask_red_cleaned = cv2.morphologyEx(mask_red, cv2.MORPH_CLOSE, kernel, iterations=3)
    mask_yellow_cleaned = cv2.morphologyEx(mask_yellow, cv2.MORPH_CLOSE, kernel, iterations=3)

    # Find Contours
    contours_red, _ = cv2.findContours(mask_red_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_yellow, _ = cv2.findContours(mask_yellow_cleaned, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    red_centroids = [get_centroid(cnt) for cnt in contours_red if get_centroid(cnt) is not None]
    yellow_centroids = [get_centroid(cnt) for cnt in contours_yellow if get_centroid(cnt) is not None]
    
    if not red_centroids or not yellow_centroids:
        continue

    # Compute Metrics Using Bounding Box Logic
    yellow_connections = [any(cv2.boundingRect(cnt)[0] in range(cv2.boundingRect(r_cnt)[0] - 15, cv2.boundingRect(r_cnt)[0] + 15) for r_cnt in contours_red) for cnt in contours_yellow]
    yellow_surrounded = [sum(cv2.boundingRect(r_cnt)[0] in range(cv2.boundingRect(cnt)[0] - 15, cv2.boundingRect(cnt)[0] + 15) for r_cnt in contours_red) >= 3 for cnt in contours_yellow]

    # Compute Distance to Nearest Red
    red_tree = KDTree(red_centroids)
    yellow_distances = [red_tree.query(yellow_point)[0] for yellow_point in yellow_centroids]
    min_distance = min(yellow_distances) if yellow_distances else 1
    max_distance = max(yellow_distances) if yellow_distances else 1
    normalized_distances = [(d - min_distance) / (max_distance - min_distance) if max_distance > min_distance else 0 for d in yellow_distances]

    # Compute AHP
    yellow_data = pd.DataFrame({
        "Jarak_ke_Merah": normalized_distances,
        "Sebagai_Penghubung": yellow_connections,
        "Dikelilingi_Merah": yellow_surrounded
    })
    yellow_data["Skor_Akhir"] = yellow_data.apply(hitung_skor_ahp, axis=1)
    
    threshold_otsu, _ = cv2.threshold((yellow_data["Skor_Akhir"].values * 255).astype(np.uint8), 0, 255, cv2.THRESH_OTSU)
    threshold_otsu /= 255  # Normalize
    yellow_data["Layak_Diubah"] = yellow_data["Skor_Akhir"] > threshold_otsu
    yellow_data["File"] = image_file
    df_list.append(yellow_data)

    # Modify Image
    image_modified = image.copy()
    for idx, yellow_contour in enumerate(contours_yellow):
        color = (0, 0, 255) if yellow_data.iloc[idx]["Layak_Diubah"] else (0, 255, 255)
        cv2.drawContours(image_modified, [yellow_contour], -1, color, -1)

    # Save Image
    modified_image_path = os.path.join(output_folder, f"modified_{image_file}")
    cv2.imwrite(modified_image_path, image_modified)

# Save AHP Results
if df_list:
    ahp_output_path = os.path.join(output_folder, "ahp_final_results.xlsx")
    pd.concat(df_list).to_excel(ahp_output_path, index=False)
    print(f"Processing complete! Results saved at {output_folder}.")
else:
    print("No valid images processed.")

Processing complete! Results saved at D:\Disertasi Bu Sussi\20-Maret\citra 5\AHP 1 Tahap.
